In [31]:
import requests
from bs4 import BeautifulSoup
import json
import base64
import urllib.parse
from datetime import datetime, timedelta


# URL 생성 함수
def create_this_week_url(base_url, start_date_str):
    # 날짜 형식 지정
    date_format = "%Y.%m.%d"
    
    # 시작 날짜 파싱
    start_date = datetime.strptime(start_date_str, date_format)
    
    # 오늘 날짜 기준 저번 주 월요일 계산
    last_week_monday = start_date - timedelta(days=start_date.weekday() + 7)
    last_week_monday_str = last_week_monday.strftime(date_format)
    
    # 인코딩된 문자열 생성
    encoded_str = f"fnct1|@@|%2Fdiet%2Fkr%2F1%2Fview.do%3Fmonday%3D{last_week_monday_str}%26week%3Dnext%26"
    
    # Base64 인코딩
    encoded_bytes = base64.b64encode(encoded_str.encode('utf-8'))
    encoded_base64_str = encoded_bytes.decode('utf-8').rstrip('=') + '='
    
    # 최종 URL 생성
    this_week_url = base_url + urllib.parse.quote(encoded_base64_str)
    return this_week_url

# 기본 URL 설정
base_url = "https://www.inha.ac.kr/kr/1073/subview.do?&enc="

# 오늘 날짜 설정
today_date_str = datetime.now().strftime("%Y.%m.%d")

# 이번 주 URL 생성
this_week_url = create_this_week_url(base_url, today_date_str)
print(f"This week URL: {this_week_url}")

# 1. 웹페이지 접근
response = requests.get(this_week_url)

# 응답 코드 확인
if response.status_code != 200:
    print(f"Failed to fetch page, status code: {response.status_code}")
else:
    # 2. HTML 파싱
    soup = BeautifulSoup(response.content, 'html.parser')

# 날짜와 식단 정보를 담을 리스트
week_data = []

# 식단 날짜 가져오기
week_range_tag = soup.select_one('.moveWeekBox strong')
if week_range_tag:
    week_range = week_range_tag.text.strip()
else:
    print("Failed to find the week range")
    week_range = "Unknown"

# 요일별 식단 정보 추출
for day_section in soup.select('.foodInfoWrap'):
    day_title_tag = day_section.find_previous_sibling('h2')
    if day_title_tag:
        day_title = day_title_tag.text.strip()

        # 각 식단별 정보 추출
        for row in day_section.select('tbody tr'):
            meal = {}
            meal['날짜'] = day_title
            meal['구분'] = row.find('th').text.strip()
            menu_cell = row.find('td')
            if menu_cell:
                # '<br>' 태그로 나눈 텍스트 리스트를 생성
                menu_items = menu_cell.decode_contents().split('<br>')
                # 각 아이템을 스트립하여 정리
                menu_items = [BeautifulSoup(item, 'html.parser').text.strip() for item in menu_items if item.strip()]
                 # 공백, 탭, 개행 문자 제거
                menu_items = [item.replace('\t', '').replace('\r', ',').replace('\n', '').replace('*','&').replace('D',' 드레싱').replace('S',' 소스') for item in menu_items]
                menu_items = [item.replace(',&', '&') for item in menu_items]
                meal['메뉴'] = ', '.join(menu_items)
                # 마지막 수정: ',&' -> '&'
            else:
                meal['메뉴'] = ""
            week_data.append(meal)

# 결과 JSON 변환
json_data = json.dumps(week_data, ensure_ascii=False, indent=4)
print(json_data)

# JSON 파일로 저장
with open('menu_data.json', 'w', encoding='utf-8') as json_file:
    json.dump(week_data, json_file, ensure_ascii=False, indent=4)


This week URL: https://www.inha.ac.kr/kr/1073/subview.do?&enc=Zm5jdDF8QEB8JTJGZGlldCUyRmtyJTJGMSUyRnZpZXcuZG8lM0Ztb25kYXklM0QyMDI0LjA2LjE3JTI2d2VlayUzRG5leHQlMjY%3D
[
    {
        "날짜": "월요일 (06.24.)",
        "구분": "조식(08:20~09:20)",
        "메뉴": "훈제오리볶음, 쌀밥, 들기름미역국,명엽채조림,와사비부추무침,배추김치,끓인누룽지"
    },
    {
        "날짜": "월요일 (06.24.)",
        "구분": "중식 백반(11:20~13:30)",
        "메뉴": "오삼불고기,잡곡밥/쌀밥,콩나물국,옥수수콘달걀찜,오이부추생채,배추김치"
    },
    {
        "날짜": "월요일 (06.24.)",
        "구분": "중식 특식(11:20~13:30)",
        "메뉴": "카레라이스&왕새우튀김&타르 소스,어묵국,타코야끼,그린샐러드&오리엔탈 드레싱,피클&단무지,배추김치"
    },
    {
        "날짜": "월요일 (06.24.)",
        "구분": "석식(17:00~18:00)",
        "메뉴": "장터국밥,잡곡밥/쌀밥,생선까스&타르타르소스,건파래볶음,고추지무침,깍두기"
    },
    {
        "날짜": "월요일 (06.24.)",
        "구분": "스낵(조식/석식)",
        "메뉴": "토핑라면"
    },
    {
        "날짜": "화요일 (06.25.)",
        "구분": "조식(08:20~09:20)",
        "메뉴": "돈육김치찌개,쌀밥,야채고로케&케찹,부들어묵볶음,미역초무침,백김치,끓인누룽지"
    },
    {
        "날짜": "화요일 (06.25.)",
        "구분": "중식 백반(1